# Gallery Example: H₂/M/1 Queue (Hyperexponential Arrivals)

This example demonstrates an H₂/M/1 queueing system:
- **Arrivals**: Hyperexponential distribution (more variable than Poisson)
- **Service**: Exponential service times
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

The Hyperexponential distribution has higher variance than exponential, representing more bursty/irregular arrivals with high variability (SCV=64).

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_hypm1():    """Create H₂/M/1 queueing model"""    model = Network('H2/M/1')        # Block 1: nodes    source = Source(model, 'mySource')    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)    sink = Sink(model, 'mySink')        # Block 2: classes    oclass = OpenClass(model, 'myClass')    # Hyperexponential arrival with mean=1 and SCV=64 (highly variable)    source.set_arrival(oclass, HyperExp.fit_mean_and_scv(1, 64))    # Exponential service with rate=2 (mean service time = 0.5)    queue.set_service(oclass, Exp(2))        # Block 3: topology    P = model.init_routing_matrix()    P.add_route(oclass, source, queue, 1.0)    P.add_route(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_hypm1()

## Theoretical Analysis for H₂/M/1

For H₂/M/1 with:
- **Arrival rate**: λ = 1 (Hyperexponential with mean=1, SCV=64)
- **Service rate**: μ = 2 (exponential)
- **Utilization**: ρ = λ/μ = 0.5

The Hyperexponential distribution characteristics:
- **Mean**: 1.0
- **Squared Coefficient of Variation**: SCV = 64 (much higher than exponential SCV = 1)
- **Variance**: 64 (very high variability)

This high arrival variability typically results in much worse performance than M/M/1.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=20)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)

In [ ]:
# Compare H₂/M/1 with M/M/1 and Er/M/1
print("\n=== Comparison with M/M/1 and Er/M/1 ===")

# Create equivalent models
def create_mm1_equivalent():
    model_mm1 = Network('M/M/1-Equivalent')
    source = Source(model_mm1, 'Source')
    queue = Queue(model_mm1, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_mm1, 'Sink')
    
    oclass = OpenClass(model_mm1, 'Class')
    source.set_arrival(oclass, Exp(1))  # SCV = 1
    queue.set_service(oclass, Exp(2))
    
    P = model_mm1.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_mm1.link(P)
    
    return model_mm1

def create_erlm1_equivalent():
    model_erl = Network('Er/M/1-Equivalent')
    source = Source(model_erl, 'Source')
    queue = Queue(model_erl, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_erl, 'Sink')
    
    oclass = OpenClass(model_erl, 'Class')
    source.set_arrival(oclass, Erlang.fit_mean_and_order(1, 5))  # SCV = 0.2
    queue.set_service(oclass, Exp(2))
    
    P = model_erl.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_erl.link(P)
    
    return model_erl

model_mm1 = create_mm1_equivalent()
model_erl = create_erlm1_equivalent()

solver_hyp = MVA(model)
solver_mm1 = MVA(model_mm1)
solver_erl = MVA(model_erl)

avg_table_hyp = solver_hyp.avg_table()
avg_table_mm1 = solver_mm1.avg_table()
avg_table_erl = solver_erl.avg_table()

# Extract key metrics
hyp_resp = float(avg_table_hyp.iloc[1, 2])
hyp_length = float(avg_table_hyp.iloc[1, 3])

mm1_resp = float(avg_table_mm1.iloc[1, 2])
mm1_length = float(avg_table_mm1.iloc[1, 3])

erl_resp = float(avg_table_erl.iloc[1, 2])
erl_length = float(avg_table_erl.iloc[1, 3])

print("Model Comparison (same λ=1, μ=2):")
print(f"Distribution | SCV  | Response Time | Queue Length")
print("-" * 50)
print(f"Erlang (k=5) | 0.20 |    {erl_resp:.4f}    |   {erl_length:.4f}")
print(f"Exponential  | 1.00 |    {mm1_resp:.4f}    |   {mm1_length:.4f}")
print(f"Hyperexp     |64.00 |    {hyp_resp:.4f}    |   {hyp_length:.4f}")

print(f"\nPerformance Impact:")
print(f"H₂/M/1 vs M/M/1: {(hyp_resp / mm1_resp):.1f}x worse response time")
print(f"Er/M/1 vs M/M/1: {(erl_resp / mm1_resp):.2f}x better response time")
print(f"Range: {(hyp_resp / erl_resp):.1f}x difference between high and low variability")

In [ ]:
# Analyze impact of arrival variability (SCV)
print("\n=== Impact of Arrival Variability ===")

def create_hyperexp_model(scv):
    """Create H₂/M/1 model with specified SCV"""
    if scv < 1:
        # Use Erlang for SCV < 1
        model_var = Network(f'Er/M/1-SCV{scv}')
        source = Source(model_var, 'Source')
        queue = Queue(model_var, 'Queue', SchedStrategy.FCFS)
        sink = Sink(model_var, 'Sink')
        
        oclass = OpenClass(model_var, 'Class')
        order = int(1.0 / scv)
        source.set_arrival(oclass, Erlang.fit_mean_and_order(1, order))
        queue.set_service(oclass, Exp(2))
    elif scv == 1:
        # Use Exponential for SCV = 1
        model_var = Network(f'M/M/1-SCV{scv}')
        source = Source(model_var, 'Source')
        queue = Queue(model_var, 'Queue', SchedStrategy.FCFS)
        sink = Sink(model_var, 'Sink')
        
        oclass = OpenClass(model_var, 'Class')
        source.set_arrival(oclass, Exp(1))
        queue.set_service(oclass, Exp(2))
    else:
        # Use Hyperexponential for SCV > 1
        model_var = Network(f'H2/M/1-SCV{scv}')
        source = Source(model_var, 'Source')
        queue = Queue(model_var, 'Queue', SchedStrategy.FCFS)
        sink = Sink(model_var, 'Sink')
        
        oclass = OpenClass(model_var, 'Class')
        source.set_arrival(oclass, HyperExp.fit_mean_and_scv(1, scv))
        queue.set_service(oclass, Exp(2))
    
    P = model_var.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_var.link(P)
    
    return model_var

# Test different SCV values
scv_values = [0.2, 0.5, 1.0, 4.0, 16.0, 64.0]

print("SCV  | Response Time | Queue Length | Performance Factor")
print("-" * 55)

baseline_resp = None
for scv in scv_values:
    model_scv = create_hyperexp_model(scv)
    solver = MVA(model_scv)
    avg_table = solver.avg_table()
    
    resp_time = float(avg_table.iloc[1, 2])
    queue_length = float(avg_table.iloc[1, 3])
    
    if baseline_resp is None and scv == 1.0:
        baseline_resp = resp_time
    
    perf_factor = resp_time / baseline_resp if baseline_resp else 1.0
    
    print(f"{scv:4.1f} |    {resp_time:.4f}    |   {queue_length:.4f}    |      {perf_factor:.2f}")

print("\nNote: Higher SCV (more variability) leads to significantly worse performance.")
print("This demonstrates the fundamental impact of variability in queueing systems.")